# Validating OCDS Records Interactively in Jupyter using LibCove.
S.A.Boyle 19/08/2021

## Requirements
* !pip install libcove
* !pip install libcoveocds

In [1]:
from libcoveocds.lib.api import context_api_transform
from libcoveocds.common_checks import common_checks_ocds
from libcoveocds.schema import SchemaOCDS
from libcoveocds.config import LibCoveOCDSConfig
from pathlib import Path
import json

## Retrieving OCDS json from URL 

In [2]:
import urllib.request

def load_ocds_from_url(ocds_url):
    """
    load_ocds_from_url: fetch and parse an ocds file from ocds_url.
    """
    with urllib.request.urlopen(ocds_url) as response:
        raw_json = response.read()
        json_data = json.loads(raw_json)    
    return json_data

def load_ocds_from_file(filename):
    raw_json = Path(filename).read_text()
    json_data = json.loads(raw_json)
    return json_data

## Validating OCDS compliance from pasted JSON fragment

In [3]:

def validate_json(json_data, context = {"file_type": 'json'}, schema_version="1.1"):
    """ 
    validate_json: validate json_data against the specified OCDS shema_version 
    """
    lib_cove_ocds_config = LibCoveOCDSConfig()
    schema_ocds = SchemaOCDS(schema_version, json_data, lib_cove_ocds_config=lib_cove_ocds_config, record_pkg=False)

    result = context_api_transform(
            common_checks_ocds(context, '/tmp', json_data, schema_ocds, api=True, cache=False)
        )
    return result

## Validating an OCDS Json file

In [4]:
filename = "tempinput/testinput.json"

def validate_json_file(filename, schema_version="1.1"):
    """ 
    validate_json_file: validate OCDS json held in filename against the specified OCDS shema_version 
    """
    json_data = load_ocds_from_file(filename)
    return validate_json(json_data, schema_version=schema_version)

result = validate_json_file(filename)

## Validating OCDS from Url

In [5]:
def validate_url(ocds_url, schema_version="1.1"):
    """ 
    validate_json_url: validate OCDS json hosted at ocds_url against the specified OCDS shema_version 
    """
    json_data = load_ocds_from_url(ocds_url)
    return validate_json(json_data, schema_version=schema_version)

result = validate_url('http://www.ppip.gov.np/api/contracts/ocds/3')

## Inspecting available sections in the result dictionary

In [6]:
result.keys()

dict_keys(['file_type', 'version_used', 'schema_url', 'extensions', 'validation_errors', 'common_error_types', 'deprecated_fields', 'releases_aggregates', 'additional_closed_codelist_values', 'additional_open_codelist_values', 'additional_checks', 'all_additional_fields', 'additional_fields', 'ocds_prefixes_bad_format'])

## Inspecting detailed results within a given results section

In [84]:
result['validation_errors']

[{'type': 'number',
  'field': 'releases/planning/budget/amount/amount',
  'description': "'amount' is not a number. Check that the value  doesn’t contain any characters other than 0-9 and dot ('.'). Number values should not be in quotes. ",
  'path': 'releases/0/planning/budget/amount/amount',
  'value': 'n/a'}]

In [113]:
result['deprecated_fields']

[{'paths': ('releases/0/buyer',),
  'explanation': ('1.1',
   'From version 1.1, organizations should be referenced by their identifier and name in a document, and address information should only be provided in the relevant cross-referenced entry in the parties section at the top level of a release.'),
  'field': 'address'},
 {'paths': ('releases/0/tender',),
  'explanation': ('1.1',
   'The single amendment object has been deprecated in favour of including amendments in an amendments (plural) array.'),
  'field': 'amendment'},
 {'paths': ('releases/0/buyer',),
  'explanation': ('1.1',
   'From version 1.1, organizations should be referenced by their identifier and name in a document, and contact point information for an organization should be provided in the relevant cross-referenced entry in the parties section at the top level of a release.'),
  'field': 'contactPoint'},
 {'paths': ('releases/0/buyer',),
  'explanation': ('1.1',
   'From version 1.1, organizations should be referenc

---

## Rendering OCDS validation results in user friendly format

In [104]:
#!pip install Jinja2

In [108]:
from jinja2 import Template
import IPython

In [121]:
def render_validation_checks(result, title="Validation Checks"):
    validation_checks_template = """
        <h2>{{ title }}</h2>
        <table class="table">
            <thead>
              <tr>
                <th>Type</th>
                <th>Field</th>
                <th>Description</th>
                <th>Path</th>
                <th>Value</th>
              </tr>
            </thead>
            <tbody>
                {% for i in data %}
                <tr>
                    <td>{{ i.type }}</td>
                    <td>{{ i.field }}</td>
                    <td>{{ i.description }}</td>
                    <td>{{ i.path }}</td>
                    <td>{{ i.value }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        """

    template = Template(validation_checks_template)
    h = template.render(data = result['validation_errors'], title=title)
    return IPython.display.HTML(h)

render_validation_checks(result)

Type,Field,Description,Path,Value
number,releases/planning/budget/amount/amount,'amount' is not a number. Check that the value doesn’t contain any characters other than 0-9 and dot ('.'). Number values should not be in quotes.,releases/0/planning/budget/amount/amount,n/a


In [128]:
def render_additional_fields(result, title="Additional Fields"):
    additional_fields_template = """
        <h2>{{ title }}</h2>
        <table class="table">
            <thead>
              <tr>
                <th>Path</th>
                <th>Field</th>
                <th>Usage Count</th>
              </tr>
            </thead>
            <tbody>
                {% for i in data %}
                <tr>
                    <td>{{ i.path }}</td>
                    <td>{{ i.field }}</td>
                    <td>{{ i.usage_count }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        """

    template = Template(additional_fields_template)
    h = template.render(data = result['additional_fields'], title=title)
    return IPython.display.HTML(h)

render_additional_fields(result)

Path,Field,Usage Count
/releases/tender,criteria,1


In [125]:
def render_deprecated_fields(result, title="Deprecated Fields"):
    deprecated_fields_template = """
        <h2>{{ title }}</h2>
        <table class="table">
            <thead>
              <tr>
                <th>Paths</th>
                <th>Field</th>
                <th>Explanation</th>
              </tr>
            </thead>
            <tbody>
                {% for i in data %}
                <tr>
                    <td>{{ i.paths }}</td>
                    <td>{{ i.field }}</td>
                    <td>{{ i.explanation }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        """

    template = Template(deprecated_fields_template)
    h = template.render(data = result['deprecated_fields'], title=title)
    return IPython.display.HTML(h)

render_deprecated_fields(result)

Paths,Field,Explanation
"('releases/0/buyer',)",address,"('1.1', 'From version 1.1, organizations should be referenced by their identifier and name in a document, and address information should only be provided in the relevant cross-referenced entry in the parties section at the top level of a release.')"
"('releases/0/tender',)",amendment,"('1.1', 'The single amendment object has been deprecated in favour of including amendments in an amendments (plural) array.')"
"('releases/0/buyer',)",contactPoint,"('1.1', 'From version 1.1, organizations should be referenced by their identifier and name in a document, and contact point information for an organization should be provided in the relevant cross-referenced entry in the parties section at the top level of a release.')"
"('releases/0/buyer',)",identifier,"('1.1', 'From version 1.1, organizations should be referenced by their identifier and name in a document, and detailed legal identifier information should only be provided in the relevant cross-referenced entry in the parties section at the top level of a release.')"
